OpenStax Data Extraction
===

Prealgebra: https://openstax.org/books/prealgebra-2e/pages/1-introduction

Algebra 1: https://openstax.org/details/books/elementary-algebra-2e

In [4]:
import bs4
import requests

In [6]:
data = requests.get("https://openstax.org/books/prealgebra-2e/pages/1-introduction")

In [14]:
html_doc = data.content.decode()
html_doc[:50]

'<!doctype html><html lang="en" prefix="og: http://'

In [15]:
soup = bs4.BeautifulSoup(html_doc, "html.parser")

In [19]:
with open("doc.html", "w") as outfile:
    outfile.write(soup.prettify())

In [21]:
soup.find_all("summary")[0].find_all("a")

[]

In [26]:
toc = soup.find_all(attrs={"class": "os-text"})
len(toc)

172

In [43]:
toc[3].parent.parent.parent.parent.parent.parent.find_all(attrs={"class": "os-text"})

[<span class="os-text" data-type="" itemprop="">Preface</span>,
 <span class="os-text" data-type="" itemprop="">Whole Numbers</span>,
 <span class="os-text" data-type="" itemprop="">Introduction</span>,
 <span class="os-text" data-type="" itemprop="">Introduction to Whole Numbers</span>,
 <span class="os-text" data-type="" itemprop="">Add Whole Numbers</span>,
 <span class="os-text" data-type="" itemprop="">Subtract Whole Numbers</span>,
 <span class="os-text" data-type="" itemprop="">Multiply Whole Numbers</span>,
 <span class="os-text" data-type="" itemprop="">Divide Whole Numbers</span>,
 <span class="os-text">Chapter Review</span>,
 <span class="os-text">Key Terms</span>,
 <span class="os-text">Key Concepts</span>,
 <span class="os-text">Exercises</span>,
 <span class="os-text">Review Exercises</span>,
 <span class="os-text">Practice Test</span>,
 <span class="os-text" data-type="" itemprop="">The Language of Algebra</span>,
 <span class="os-text" data-type="" itemprop="">Introduct